In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
site_name = ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E']

#####This is to make a simple dataframe with stats from launchsites

#launchsites = spacex_df.groupby(['Launch Site','class']).size().to_frame().reset_index()
#launchsites.rename(columns={0:'count'},inplace=True)
#total0=launchsites[['count']].sum(axis=1).where(launchsites['class']==0, 0).sum()
#total1=launchsites[['count']].sum(axis=1).where(launchsites['class']==1, 0).sum()
#allsites={'Launch Site':['All Sites','All Sites'], 'class': [0,1], 'count':[total0,total1]}
#allsites=pd.DataFrame(allsites)
#launchsites=launchsites.append(allsites)


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div([
                                       # Create an division for adding dropdown helper text for choosing year
                                        html.Div(
                                            [
                                            html.H2('Select a Launch Site:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='site-dropdown', 
                                            options=[{'label': 'All Sites', 'value':'All Sites'},
                                                    {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'}],
                                            value='All Sites',
                                            placeholder="Select a Launch Site here", searchable=True,
                                            style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}
                                            ),
                                          ]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000,
                                step=None, marks={0: '0', 1000: '1000', 2000: '2000', 3000: '3000',
                                4000: '4000', 5000: '5000', 6000: '6000', 7000: '7000', 8000: '8000', 
                                9000: '9000', 10000: '10000'},
                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))
def get_pie(site_selected):

    if site_selected == 'All Sites':
        pie_data = spacex_df.groupby(['Launch Site','class']).size().groupby(level=1)
        success=pie_data.get_group(1)
        unsuccess=pie_data.get_group(0)
        fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Succesful', 'Unsuccesful'])
        fig.add_trace(go.Pie(labels=site_name, values=success), row=1, col=1)
        fig.add_trace(go.Pie(labels=site_name, values=unsuccess), row=1, col=2)

        fig.update_layout(title_text='Launches by site')
        return fig
    else:
        launchsite =  spacex_df[spacex_df['Launch Site']==(site_selected)]
        pie_data = launchsite['class'].value_counts(sort=False)
        names=['unsuccesful','succesful']
        fig = px.pie(pie_data, values='class', names=names, color=names)
        fig.update_layout(title='Total Launches For Site '+str(site_selected))
        return fig
#, marker_colors=['blue','orange']
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_scatter(site_selected,payload_selected):

    spacex_df['Booster Version Category'] = spacex_df['Booster Version'].str.split(' ').str[1]
    if site_selected == 'All Sites':
        scatter_data = spacex_df.loc[spacex_df['Payload Mass (kg)'] > payload_selected[0]]
        scatter_data = scatter_data.loc[scatter_data['Payload Mass (kg)'] < payload_selected[1]]

        if len(scatter_data)==0:
            fig = px.scatter(scatter_data, x='Payload Mass (kg)', y='class')
        else:
            fig = px.scatter(scatter_data, x='Payload Mass (kg)', y='class', color= 'Booster Version Category')
        fig.update_layout(title='Correlation Between Payload and Success for '+str(site_selected),
         xaxis_title='Payload Mass (kg)', yaxis_title='class')
        return fig
    else:
        launchsite =  spacex_df[spacex_df['Launch Site']==(site_selected)]
        scatter_data = launchsite.loc[launchsite['Payload Mass (kg)'] > payload_selected[0]]
        scatter_data = scatter_data.loc[scatter_data['Payload Mass (kg)'] < payload_selected[1]]

        if len(scatter_data)==0:
            fig = px.scatter(scatter_data, x='Payload Mass (kg)', y='class')
        else:
            fig = px.scatter(scatter_data, x='Payload Mass (kg)', y='class', color= 'Booster Version Category')
        fig.update_layout(title='Correlation Between Payload and Success for '+str(site_selected),
         xaxis_title='Payload Mass (kg)', yaxis_title='class')
        return fig



# Run the app
if __name__ == '__main__':
    app.run_server()

In [1]:
test2

NameError: name 'test2' is not defined